In [32]:
import pandas as pd
from sqlalchemy import create_engine
import nltk
from nltk.tokenize import RegexpTokenizer
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns
import time 
from nltk.stem.snowball import SnowballStemmer
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer 
import string
from sklearn.metrics import classification_report, accuracy_score, mean_squared_error, auc
from sklearn.metrics import roc_curve, make_scorer, precision_score, recall_score, f1_score
from sklearn import tree
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.model_selection import KFold
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from matplotlib.legend_handler import HandlerLine2D 
import scipy
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from vecstack import stacking
from sklearn.model_selection import GridSearchCV

In [33]:
stopwords = set(['ourselves', 'between', 'but', 'again','there', 'about', 'once', \
                 'during', 'out', 'very', 'having', 'with', 'they', 'own', 'an',\
                 'be', 'some', 'for', 'do', 'its', 'such', 'into', 'of', 'most', 
                 'itself', 'other', 'off', 'is', 's', 'am', 'or', 'who', 'as', 'from',\
                 'each', 'the', 'themselves', 'until', 'below', 'are', 'we', 'these', 'your',\
                 'through', 'don', 'nor', 'me', 'were', 'more',\
                 'this', 'down', 'should', 'our', 'their', 'while', 'above', \
                 'both', 'up', 'to', 'ours', 'had', 'all', 'no', 'when'\
                 , 'at', 'any', 'before', 'them', 'same', 'and', 'been', 'have', \
                 'in', 'will', 'on', 'does', 'yourselves', 'then', 'that', 'because', \
                 'what', 'over', 'why', 'so', 'can', 'did', 'not', 'now', 'under', \
                 'has', 'just', 'where', 'too', 'only', 'myself', 'which', 'those'\
                 , 'after', 'few', 'whom', 't', 'being', 'if', 'theirs', 'my', 'against',\
                 'a', 'by', 'doing', 'it', 'how','b','the', 'you', 'further', 'href', \
                 'was', 'here', 'than','you'])

In [34]:
d = pd.read_csv("sample_data.csv")

In [35]:
scorer={'accuracy': make_scorer(accuracy_score),
        'f1_score': make_scorer(f1_score),
        'precision': make_scorer(precision_score, pos_label=1, average='binary'),
        'recall': make_scorer(recall_score), 
        'mean_squared_error': make_scorer(mean_squared_error)}

In [36]:
X_train, X_test, y_train, y_test= train_test_split(d.loc[:, 'text_info'],\
                                                       d.loc[:, 'bechdal_int'], test_size = 0.2)
v2 = TfidfVectorizer(analyzer='word',use_idf=True, ngram_range=(2,2), \
                         token_pattern= r'\w{1,}', stop_words=stopwords, sublinear_tf=True)
train_X = v2.fit_transform(X_train)
test_X = v2.transform(X_test)

# Stacks

In [85]:
dTree = tree.DecisionTreeClassifier(max_depth = 4)
nb = MultinomialNB(alpha =0.1)
rf = RandomForestClassifier(max_depth=4, random_state=0, n_estimators = 50)
ab = AdaBoostClassifier(dTree, n_estimators =50)
lr = LogisticRegression(solver = 'lbfgs')


models = [dTree,nb,rf,ab,lr]

In [86]:
r1_train, r1_test = stacking(models,train_X, y_train, test_X, regression=False, \
                           mode='oof_pred_bag', save_dir=None, metric=accuracy_score, \
                           n_folds=5, stratified=True,shuffle=True,verbose=2)

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [5]

model  0:     [DecisionTreeClassifier]
    fold  0:  [0.66666667]
    fold  1:  [0.55555556]
    fold  2:  [0.50000000]
    fold  3:  [0.71428571]
    fold  4:  [1.00000000]
    ----
    MEAN:     [0.68730159] + [0.17399606]
    FULL:     [0.67500000]

model  1:     [MultinomialNB]
    fold  0:  [0.55555556]
    fold  1:  [0.55555556]
    fold  2:  [0.37500000]
    fold  3:  [0.28571429]
    fold  4:  [0.42857143]
    ----
    MEAN:     [0.44007937] + [0.10475289]
    FULL:     [0.45000000]

model  2:     [RandomForestClassifier]
    fold  0:  [0.55555556]
    fold  1:  [0.55555556]
    fold  2:  [0.62500000]
    fold  3:  [0.57142857]
    fold  4:  [0.57142857]
    ----
    MEAN:     [0.57579365] + [0.02560677]
    FULL:     [0.57500000]

model  3:     [AdaBoostClassifier]
    fold  0:  [0.66666667]
    fold  1:  [0.55555556]
    fold  2:  [0.50000000]
    fo

In [87]:
r1_train, r1_test = stacking(top_3_models,train_X, y_train, test_X, regression=False, \
                           mode='oof_pred_bag', save_dir=None, metric=accuracy_score, \
                           n_folds=5, stratified=True,shuffle=True,verbose=2)

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [RandomForestClassifier]
    fold  0:  [0.55555556]
    fold  1:  [0.55555556]
    fold  2:  [0.62500000]
    fold  3:  [0.57142857]
    fold  4:  [0.57142857]
    ----
    MEAN:     [0.57579365] + [0.02560677]
    FULL:     [0.57500000]

model  1:     [LogisticRegression]
    fold  0:  [0.55555556]
    fold  1:  [0.55555556]
    fold  2:  [0.62500000]
    fold  3:  [0.57142857]
    fold  4:  [0.57142857]
    ----
    MEAN:     [0.57579365] + [0.02560677]
    FULL:     [0.57500000]

model  2:     [AdaBoostClassifier]
    fold  0:  [0.66666667]
    fold  1:  [0.55555556]
    fold  2:  [0.50000000]
    fold  3:  [0.00000000]
    fold  4:  [0.71428571]
    ----
    MEAN:     [0.48730159] + [0.25533363]
    FULL:     [0.50000000]



In [88]:
model1 = AdaBoostClassifier(dTree, n_estimators = 50)
    
model1 = model1.fit(r1_train, y_train)
y_pred1 = model1.predict(r1_test)
accuracy_score(y_test, y_pred1)

0.2727272727272727

In [89]:
top_3_models  = [rf,lr,ab] 

In [90]:
r2_train, r2_test = stacking(top_3_models,train_X, y_train, test_X, regression=False, \
                           mode='oof_pred_bag', save_dir=None, metric=accuracy_score, \
                           n_folds=5, stratified=True,shuffle=True,verbose=2)

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [RandomForestClassifier]
    fold  0:  [0.55555556]
    fold  1:  [0.55555556]
    fold  2:  [0.62500000]
    fold  3:  [0.57142857]
    fold  4:  [0.57142857]
    ----
    MEAN:     [0.57579365] + [0.02560677]
    FULL:     [0.57500000]

model  1:     [LogisticRegression]
    fold  0:  [0.55555556]
    fold  1:  [0.55555556]
    fold  2:  [0.62500000]
    fold  3:  [0.57142857]
    fold  4:  [0.57142857]
    ----
    MEAN:     [0.57579365] + [0.02560677]
    FULL:     [0.57500000]

model  2:     [AdaBoostClassifier]
    fold  0:  [0.66666667]
    fold  1:  [0.66666667]
    fold  2:  [0.50000000]
    fold  3:  [0.00000000]
    fold  4:  [1.00000000]
    ----
    MEAN:     [0.56666667] + [0.32659863]
    FULL:     [0.57500000]



In [91]:
model2 = AdaBoostClassifier(dTree, n_estimators = 50)
    
model2 = model2.fit(r2_train, y_train)
y_pred2 = model2.predict(r2_test)
accuracy_score(y_test, y_pred2)

0.5454545454545454

In [92]:
r_models  = [rf,nb,ab]

In [93]:
r3_train, r3_test = stacking(r_models,train_X, y_train, test_X, regression=False, \
                           mode='oof_pred_bag', save_dir=None, metric=accuracy_score, \
                           n_folds=5, stratified=True,shuffle=True,verbose=2)

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [RandomForestClassifier]
    fold  0:  [0.55555556]
    fold  1:  [0.55555556]
    fold  2:  [0.62500000]
    fold  3:  [0.57142857]
    fold  4:  [0.57142857]
    ----
    MEAN:     [0.57579365] + [0.02560677]
    FULL:     [0.57500000]

model  1:     [MultinomialNB]
    fold  0:  [0.55555556]
    fold  1:  [0.55555556]
    fold  2:  [0.37500000]
    fold  3:  [0.28571429]
    fold  4:  [0.42857143]
    ----
    MEAN:     [0.44007937] + [0.10475289]
    FULL:     [0.45000000]

model  2:     [AdaBoostClassifier]
    fold  0:  [0.66666667]
    fold  1:  [0.55555556]
    fold  2:  [0.50000000]
    fold  3:  [0.00000000]
    fold  4:  [1.00000000]
    ----
    MEAN:     [0.54444444] + [0.32279642]
    FULL:     [0.55000000]



In [94]:
model3 = AdaBoostClassifier(dTree, n_estimators = 50)
    
model3 = model3.fit(r3_train, y_train)
y_pred3 = model3.predict(r3_test)
accuracy_score(y_test, y_pred3)

0.45454545454545453

In [95]:
two_models  = [dTree,nb]

In [96]:
train2, test2 = stacking(two_models,train_X, y_train, test_X, regression=False, \
                           mode='oof_pred_bag', save_dir=None, metric=accuracy_score, \
                           n_folds=5, stratified=True,shuffle=True,verbose=2)

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [2]

model  0:     [DecisionTreeClassifier]
    fold  0:  [0.66666667]
    fold  1:  [0.66666667]
    fold  2:  [0.50000000]
    fold  3:  [0.00000000]
    fold  4:  [0.71428571]
    ----
    MEAN:     [0.50952381] + [0.26496053]
    FULL:     [0.52500000]

model  1:     [MultinomialNB]
    fold  0:  [0.55555556]
    fold  1:  [0.55555556]
    fold  2:  [0.37500000]
    fold  3:  [0.28571429]
    fold  4:  [0.42857143]
    ----
    MEAN:     [0.44007937] + [0.10475289]
    FULL:     [0.45000000]



In [97]:
simp_model = dTree
    
simp_model = simp_model.fit(train2, y_train)
y_pred_simp2 = simp_model.predict(test2)
accuracy_score(y_test, y_pred_simp2)

0.2727272727272727